#### 03_supp_traj_measures.ipynb
In this script, we will visualize differences in mousetracking trajectory across conditions (as an index of decision uncertainty).

In [1]:
import os 
import sys 
import numpy as np 
import pandas as pd
import importlib # allows dyamic refreshing of functions
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ridgeplot import ridgeplot
from plotly.subplots import make_subplots
from glob import glob


%autosave 180

Autosaving every 180 seconds


In [2]:
# Set up directories 
currPath = os.getcwd()
rootPath = os.path.abspath(os.path.join(currPath, os.pardir, os.pardir))
mouse_dataPath = os.path.join(rootPath, 'data/processed/trajectories')

# Grab the subject mouse dirs
sub_files = glob(os.path.join(mouse_dataPath, 'sub-[0-9][0-9]_*')) + glob(os.path.join(mouse_dataPath, 'sub-[0-9][0-9][0-9]_*'))
assert len(sub_files) == 83 # make sure all subs are there

# Also, import some functions 
sys.path.insert(1, os.path.join(rootPath, 'notebooks/analyses/utils'))
import mouse_func; importlib.reload(mouse_func)
from mouse_func import *

In [3]:
dist_master = [] # for group dataframe of distances
coords_master = [] # for group dataframe of time-normalized coordinates

# For each subject... 
for s in sub_files: 
    # Import the mouse file 
    sub_data = pd.read_csv(s)
    
    # Extract only paired items that were correctly sorted during the decision phase
    correct_pair = sub_data[(sub_data['both_enc_acc'] == 1) & (sub_data['paired_status'] == 'paired')].copy()

    # Order the pairmates 
    for stim_name in correct_pair['stimulus_name']: 
        stim_df = correct_pair[correct_pair['stimulus_name'] == stim_name].copy()
        stim_df = stim_df.sort_values(by=['Trial Number'])

        # Assign pairmate order 
        correct_pair.loc[stim_df.index.values[0], 'pairmate'] = 'first'
        correct_pair.loc[stim_df.index.values[1], 'pairmate'] = 'second'

    # Remove trials that have no mouse data 
    correct_pair = correct_pair[~correct_pair['x_pos'].isna()]
    
    # Get necessary columns only 
    correct_pair = correct_pair[['sub_label', 'Trial Number', 'condition', 
                                 'stimulus_id', 'Reaction Time', 'both_enc_acc', 
                                 'x_pos', 'y_pos', 'time_trajectories', 'pairmate']]
    
    # Create columns for interpolated coordinates 
    correct_pair['x_interp'] = np.empty(len(correct_pair), dtype=object); correct_pair['x_interp'] = correct_pair['x_interp'].astype('object')
    correct_pair['y_interp'] = np.empty(len(correct_pair), dtype=object); correct_pair['y_interp'] = correct_pair['y_interp'].astype('object')

    # Reset indices of dataframe
    correct_pair.reset_index(drop=True, inplace=True)
    coords_master.append(correct_pair)


    # Gather time normalized trajectories...
    for i in correct_pair.index.values: 
        # Extract the original coords and times
        x_coord = correct_pair.loc[i, 'x_pos'].split(' '); x_coord = np.array(x_coord).astype(float)
        y_coord = correct_pair.loc[i, 'y_pos'].split(' '); y_coord = np.array(y_coord).astype(float)
        time_traj = correct_pair.loc[i, 'time_trajectories'].split(' '); time_traj = np.array(time_traj).astype(float)
        
        # Set up time steps and the time arrays
        n_time_steps = 101

        # Normalize x and y coordinates
        x_normalized, y_normalized = time_normalize(x_coord, y_coord, n_time_steps)

        # Place them in our dataframe 
        correct_pair.at[i, 'x_interp'] = list(x_normalized); correct_pair.at[i, 'y_interp'] = list(y_normalized)

    # Gather euclidean distance from the start at each timepoint, and then add to dataframe
    for idx in correct_pair.index.values: 
        # Get the normalized x and y coordinates
        x_coord_arr = np.array(correct_pair.loc[idx, 'x_interp'])
        y_coord_arr = np.array(correct_pair.loc[idx, 'y_interp'])

        coord_dist_arr = dist_from_end(x_coord_arr, y_coord_arr)

        # Then, create a df
        coord_dist_df = pd.DataFrame([correct_pair.loc[idx, 'sub_label'], 
                                      correct_pair.loc[idx, 'condition'], 
                                      correct_pair.loc[idx, 'Trial Number'], 
                                      correct_pair.loc[idx, 'pairmate']]).T
        coord_dist_df = pd.DataFrame(np.repeat(coord_dist_df.values, len(coord_dist_arr), axis=0))
        coord_dist_df.columns = ['sub', 'condition', 'trial_num', 'pairmate'] # rename columns 

        # Place distances and timepoints in
        coord_dist_df['distance'] = coord_dist_arr
        coord_dist_df['timepoint'] = np.arange(1, len(coord_dist_df)+1)
        dist_master.append(coord_dist_df)

In [4]:
coords_master = pd.concat(coords_master)
coords_master_second = coords_master[coords_master['pairmate'] == 'second'].copy()
coords_master_second.reset_index(drop=True, inplace=True)

In [5]:
comp_goal_trials = []
unique_goal_trials = []

# For each idx and row... 
for idx, row in coords_master_second.iterrows():
    
    # create dict object (and convert coordinates list into array)
    trial_data = {
        'trial_id': f"{row['sub_label']}_{row['Trial Number']}",
        'subject_id': row['sub_label'],
        'x': np.array(row['x_interp']),  
        'y': np.array(row['y_interp'])}
    
    # gather trial data dependent on condition 
    if row['condition'] == 'competing':  
        comp_goal_trials.append(trial_data)
    elif row['condition'] == 'unique': 
        unique_goal_trials.append(trial_data)

In [6]:
# Build dictionaries with cumulative max x-deviation
data_dict_uni_xdev = {f"{i*10}-{(i+1)*10}%": [] for i in range(10)}
data_dict_comp_xdev = {f"{i*10}-{(i+1)*10}%": [] for i in range(10)}

# Link each trial (by condition) with corresponding proportion of trajectory (bin)
# Unique goal trials 
for trial in unique_goal_trials:
    x_dev_values = get_cumulative_max_x_deviation(trial['x'])
    
    n_points = len(x_dev_values)
    for i, xdev in enumerate(x_dev_values):
        time_pct = int((i / n_points) * 10)
        if time_pct == 10:
            time_pct = 9
        bin_label = f"{time_pct*10}-{(time_pct+1)*10}%"
        data_dict_uni_xdev[bin_label].append(xdev)

# Competing goal trials
for trial in comp_goal_trials:
    x_dev_values = get_cumulative_max_x_deviation(trial['x'])
    
    n_points = len(x_dev_values)
    for i, xdev in enumerate(x_dev_values):
        time_pct = int((i / n_points) * 10)
        if time_pct == 10:
            time_pct = 9
        bin_label = f"{time_pct*10}-{(time_pct+1)*10}%"
        data_dict_comp_xdev[bin_label].append(xdev)

In [ ]:
# Plot: Ridgeplot for Unique condition 
# Flatten values (for ridgeplot function)
data_dict_uni_xdev_flat = {}
for key, values in data_dict_uni_xdev.items():
    flat_values = np.array(values).flatten()  # Force flatten
    data_dict_uni_xdev_flat[key] = flat_values

# Assign variables
samples_low = list(data_dict_uni_xdev_flat.values())
labels = list(data_dict_uni_xdev_flat.keys())

fig_unique = ridgeplot(
    samples=samples_low,
    labels=labels,
    colorscale='turbo',
    bandwidth=0.05, 
    colormode = 'mean-minmax'
)

fig_unique.update_layout(
    title="Unique Goal - Cumulative Curvature Over Time",
    xaxis_title="Cumulative deviation (x-axis) from the straight line trajectory",
    yaxis_title="Decision progress (%)", 
    font=dict(family="Avenir", size=14),
    paper_bgcolor="white",
    plot_bgcolor="white",
    showlegend=False,
    margin=dict(t=60, l=60, r=30, b=50),
    xaxis=dict(
        showline=False,
        linewidth=0,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=False,
        ticks="outside",
        showticklabels=True),
    yaxis=dict(
        showline=False,
        linewidth=0,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=False,
        ticks="",
        showticklabels=True))

fig_unique.show()
fig_unique.write_image(os.path.join(rootPath, 'visualizations/04_cumulative_xdev_unique_condition.png'), 
                       scale=3, width=650, height=600) # save image as a png file

In [ ]:
# Plot: Ridgeplot for Unique condition 
# Flatten values (for ridgeplot function)
data_dict_comp_xdev_flat = {}
for key, values in data_dict_comp_xdev.items():
    flat_values = np.array(values).flatten()  # Force flatten
    data_dict_comp_xdev_flat[key] = flat_values

# Assign variables
samples_high = list(data_dict_comp_xdev_flat.values())
labels = list(data_dict_comp_xdev_flat.keys())

fig_comp = ridgeplot(
    samples=samples_high,
    labels=labels,
    colorscale='turbo',
    bandwidth=0.05, 
    colormode = 'mean-minmax')

fig_comp.update_layout(
    title="Competing Goal - Cumulative Curvature Over Time",
    xaxis_title="Cumulative deviation (x-axis) from the straight line trajectory",
    yaxis_title="Decision progress (%)", 
    font=dict(family="Avenir", size=14),
    paper_bgcolor="white",
    plot_bgcolor="white",
    showlegend=False,
    margin=dict(t=60, l=60, r=30, b=50),
    xaxis=dict(
        showline=False,
        linewidth=0,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=False,
        ticks="outside",
        showticklabels=True),
    yaxis=dict(
        showline=False,
        linewidth=0,
        gridcolor="lightgray",
        gridwidth=0.5,
        zeroline=False,
        ticks="",
        showticklabels=True))

fig_comp.show()
fig_comp.write_image(os.path.join(rootPath, 'visualizations/05_cumulative_xdev_competing_condition.png'), 
                       scale=3, width=650, height=600) # save image as a png file